In [ ]:
import pandas as pd

data_frame = pd.read_excel('data_case_study.xlsx')

li = data_frame['Straße'].drop_duplicates().tolist()

list_full_address = [e + ', Münster' for e in li] #

delivery_point = list_full_address[0]

list_full_address.append(delivery_point)

print(list_full_address)

In [ ]:
import requests

api_file = open("apikey.txt","r")
api_key = api_file.read()
api_file.close()

url = "https://maps.googleapis.com/maps/api/directions/json?"

routes = []

for i in range(len(list_full_address)-1):
    route = requests.get(
                           url 
                         + "origin=" 
                         + list_full_address[i]
                         + "&destination=" 
                         + list_full_address[i+1]
                         + "&key=" 
                         + api_key
                         )
    routes.append(route)

routes = [e.json() for e in routes]

In [ ]:
paths_polyline = [e['routes'][0]['overview_polyline']['points'] for e in routes]

lats_and_lngs = [(e['routes'][0]['legs'][0]['start_location']['lat'],
                  e['routes'][0]['legs'][0]['start_location']['lng'])
                for e in routes]

lats,lngs = zip(*lats_and_lngs)

import gmplot

gmap = gmplot.GoogleMapPlotter(sum(lats)/len(lats), sum(lngs)/len(lngs), 13, apikey = api_key)

import polyline

for i in range(len(paths_polyline)):

    path = zip(*polyline.decode(paths_polyline[i]))

    gmap.plot(*path, edge_width = 3, color = 'blue', alpha = 0.5)

gmap.scatter(lats, lngs, color = 'red', size = 40, marker = False)

gmap.marker(lats_and_lngs[0][0], lats_and_lngs[0][1], color = 'red')

for i in range(len(lats)):
    
    gmap.text(lats[i]-0.0007, lngs[i], str(i))

gmap.draw('gmap5.html')

from IPython.display import IFrame

url = 'gmap5.html'

IFrame(url, width = 800, height = 800)

In [ ]:
from prettytable import PrettyTable

route_table = PrettyTable(['No.','Address','Distance to...(km)','Est. travel time','Real time spent'])

fz = data_frame.drop_duplicates(subset=['Straße'])['GPS-Fahrzeit'].tolist()

route_table.add_row([0,delivery_point,'-','-','-'])

from time import gmtime
from time import strftime

for i in range(len(routes)):
    route_table.add_row([
                    i+1,
                    list_full_address[i+1].replace(', Münster',''), 
                    round(routes[i]["routes"][0]["legs"][0]["distance"]["value"]/1000,2),
                    strftime("%H:%M:%S", gmtime(routes[i]["routes"][0]["legs"][0]["duration"]['value'])),
                    fz[i+1 if i<25 else 0]
                    ])
    
distance = 0

for i in range(len(routes)):
    distance += routes[i]["routes"][0]["legs"][0]["distance"]["value"]

distance /= 1000
    
route_table.add_row(['Sum', '-', distance, '-', '-'])

route_table.align = 'l'

print(route_table)